<a href="https://colab.research.google.com/github/Ananth09/Pixel-Playground/blob/main/MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
import os
from PIL import Image

In [ ]:
data_dir = '/content/drive/MyDrive/img_class/'

#parameters
batch_size = 17
image_size = (224, 224)
num_epochs = 31

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    validation_split = 0.15

)

In [ ]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

Found 1527 images belonging to 2 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 268 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)


9406464/9406464 [==============================] - 1s 0us/step


In [ ]:

flatten_layer = Flatten()(base_model.layers[-1].output)
output_layer=Dense(1, activation = 'sigmoid')(flatten_layer)
model = Model(inputs=base_model.input, outputs=output_layer)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator,
    verbose=1
)

Epoch 1/31
90/90 [==============================] - 66s 339ms/step - loss: 0.1892 - accuracy: 0.9293 - val_loss: 1.2791 - val_accuracy: 0.6716
Epoch 2/31
90/90 [==============================] - 29s 323ms/step - loss: 0.0702 - accuracy: 0.9790 - val_loss: 1.0435 - val_accuracy: 0.7313
Epoch 3/31
90/90 [==============================] - 30s 328ms/step - loss: 0.0720 - accuracy: 0.9745 - val_loss: 1.6720 - val_accuracy: 0.6231
Epoch 4/31
90/90 [==============================] - 29s 319ms/step - loss: 0.0525 - accuracy: 0.9836 - val_loss: 0.7165 - val_accuracy: 0.8134
Epoch 5/31
90/90 [==============================] - 29s 323ms/step - loss: 0.0418 - accuracy: 0.9856 - val_loss: 0.6001 - val_accuracy: 0.8619
Epoch 6/31
90/90 [==============================] - 29s 322ms/step - loss: 0.0175 - accuracy: 0.9921 - val_loss: 0.2238 - val_accuracy: 0.8955
Epoch 7/31
90/90 [==============================] - 29s 322ms/step - loss: 0.0442 - accuracy: 0.9915 - val_loss: 0.4628 - val_accuracy: 0.8843

In [ ]:
model.save('mobilenetv2_2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Evaluate the model on the validation dataset
val_loss, val_accuracy = model.evaluate(val_generator, verbose=1)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")

16/16 [==============================] - 4s 235ms/step - loss: 0.1866 - accuracy: 0.9478
Validation Loss: 0.1866
Validation Accuracy: 94.78%


In [ ]:
from tensorflow import keras
model = keras.models.load_model('mobilenetv2.h5')

In [ ]:
import os
from PIL import Image

# Define the folder path
folder_path = '/content/drive/MyDrive/l(borderwhite)/'  # Replace with the actual path to your image folder

# Initialize counters and a list to store paths
lifestyle_count = 0
plain_background_count = 0
plain_background_paths = []
lifestyle_img_paths=[]  # List to store paths of plain_background images
predict_p=[]
predict_l=[]
# Loop through files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg'):
        image_path = os.path.join(folder_path, filename)

        # Load and preprocess the image
        custom_image = load_img(image_path, target_size=(224, 224))
        custom_image = img_to_array(custom_image)
        custom_image = custom_image / 255.0
        custom_image = custom_image.reshape(1, 224, 224, 3)

        # Predict using your model
        prediction = model.predict(custom_image)

        # Interpret the prediction
        if prediction > 0.5:
            result = "plain_background"
            plain_background_count += 1
            plain_background_paths.append(image_path)  # Add the path to the list
            predict_p.append(prediction)

        else:
            result = "lifestyle"
            lifestyle_count += 1
            lifestyle_img_paths.append(image_path)
            predict_l.append(prediction)
print(f"Number of 'lifestyle' images: {lifestyle_count}")
print(f"Number of 'plain_background' images: {plain_background_count}")

print("lifestyle treshold")
for tl in predict_l:
  print(tl)

print("\n\n\nplain treshold")
for pl in predict_p:
  print(pl)
# # Print the paths of 'plain_background' images
print("Paths of 'plain_background' images:")
for path in plain_background_paths:
    print(path)

print("\nPaths of lifestyle images")
for p in lifestyle_img_paths:
  print(p)


1/1 [==============================] - 0s 26ms/step
Number of 'lifestyle' images: 7
Number of 'plain_background' images: 0
lifestyle treshold
[[5.047344e-15]]
[[2.712993e-07]]
[[4.3091674e-19]]
[[3.4130202e-20]]
[[4.5138968e-10]]
[[0.03138745]]
[[2.592843e-15]]



plain treshold
Paths of 'plain_background' images:

Paths of lifestyle images
/content/drive/MyDrive/l(borderwhite)/a3b1a2aLuminox-PH-W-0320_4.jpg
/content/drive/MyDrive/l(borderwhite)/6a654b5th1791849_4.jpg
/content/drive/MyDrive/l(borderwhite)/e33aebaP1006_4.jpg
/content/drive/MyDrive/l(borderwhite)/912cb86BKPDSS304_4.jpg
/content/drive/MyDrive/l(borderwhite)/MP000000000855154_437Wx649H_202212171734511.jpg
/content/drive/MyDrive/l(borderwhite)/1c19c22SPRKL-BLK-BLK_5.jpg
/content/drive/MyDrive/l(borderwhite)/0dc4925JWBS575_1.jpg


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

custom_image_path = '/content/drive/MyDrive/predicted_lifestyle/MP000000003336728_437Wx649H_20180629211849.jpeg'
custom_image = load_img(custom_image_path, target_size=(224, 224))
custom_image = img_to_array(custom_image)
custom_image = custom_image / 255.0  # Normalize pixel values
custom_image = custom_image.reshape(1, 224, 224, 3)  # Reshape for model input


In [ ]:
# Make predictions
prediction = model.predict(custom_image)
print(prediction)
# Interpret the prediction
if prediction > 0.5:
    result = "lifestyle"
else:
    result = "plain_background"

print(f"The predicted class is: {result}")


1/1 [==============================] - 8s 8s/step
[[9.203456e-07]]
The predicted class is: plain_background
